In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
import time

In [2]:
# Ipreparametri dell'envrionment
hp_reset_noise_scale= 0.06976748570068636 # scala del rumore quando l'ambiente viene resettato 
hp_forward_reward_weight = 1.7031840270142826 # peso del reward per il movimento in avanti
hp_ctrl_cost_weight = 1.3736559851030032 # peso del reward per il controllo
hp_healthy_reward = 2.370527088282008 # reward per la salute

hp_contact_cost_weight = 5.099569789843523e-05
healthy_z = (0.27002630801618377, 1.2006270382609852)
contact_force = (-1.1373782536958372, 0.9183153315908629)

In [3]:
env_render = gym.make(
    "Ant-v5",
    reset_noise_scale=hp_reset_noise_scale,
    forward_reward_weight=hp_forward_reward_weight,
    ctrl_cost_weight=hp_ctrl_cost_weight,
    healthy_reward=hp_healthy_reward,
    contact_cost_weight=hp_contact_cost_weight,
    healthy_z_range=healthy_z,
    contact_force_range=contact_force,
    render_mode='human'
)

In [4]:
env_render.training = False # Setta l'environment in modalità di valutazione
env_render.norm_reward = False # Disabilita la normalizzazione della reward. Questo è importante per valutare correttamente il modello.

In [5]:
model = PPO.load("ppo_Ant_model")  # Carica il modello salvato
obs, _ = env_render.reset()
done = False
while not done:
    action, _ = model.predict(obs)  # Use obs directly
    obs, _, done, _, _ = env_render.step(action)  # Unpack the tuple correctly
    env_render.render()
    time.sleep(0.01)
env_render.close()

2025-02-19 22:38:07.361 Python[86497:4024505] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-19 22:38:07.361 Python[86497:4024505] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 